In [2]:
# Cell [0]
# ======================================
# 1. Install/Import Required Libraries
# ======================================

import pandas as pd
from google_play_scraper import Sort, reviews
import subprocess
from tqdm import tqdm

In [3]:
# Cell [1]
# ======================================
# 2. Define Parameters
# ======================================

# Package name of the BMW app on Google Play:
app_id = "de.bmw.connected.mobile20.row"

# Ollama model
ollama_model_name = "gemma3:12b"

In [3]:
# Cell [2]
# ======================================
# 3. Fetch Reviews from Google Play Store
# ======================================

# Define languages to fetch (just language codes and labels)
languages = [
    ('en', 'English'),
    ('de', 'German'),
    ('fr', 'French'),
    ('it', 'Italian'),
    ('es', 'Spanish'),
    ('nl', 'Dutch'),
    ('sv', 'Swedish'),
    ('da', 'Danish'),
    ('no', 'Norwegian'),
    ('fi', 'Finnish'),
    ('pl', 'Polish'),
    ('cs', 'Czech'),
    ('pt', 'Portuguese'),
    ('zh', 'Chinese'),
    ('ja', 'Japanese'),
    ('ko', 'Korean'),
    ('ar', 'Arabic'),
    ('tr', 'Turkish'),
    ('ru', 'Russian'),
    ('he', 'Hebrew'),
    ('th', 'Thai'),
    ('vi', 'Vietnamese'),
    ('hi', 'Hindi'),
    ('el', 'Greek'),
    ('hu', 'Hungarian'),
    ('ro', 'Romanian'),
    ('sk', 'Slovak'),
    ('bg', 'Bulgarian'),
    ('hr', 'Croatian'),
    ('sr', 'Serbian'),
    ('uk', 'Ukrainian'),
    ('id', 'Indonesian'),
    ('ms', 'Malay'),
    ('fa', 'Persian'),
    ('ur', 'Urdu'),
    ('bn', 'Bengali'),
    ('ta', 'Tamil'),
    ('te', 'Telugu'),
    ('ml', 'Malayalam'),
    ('et', 'Estonian'),
    ('lv', 'Latvian'),
    ('lt', 'Lithuanian'),
    ('sl', 'Slovenian')
]

# Initialize empty list to store all reviews
all_reviews = []

# Fetch reviews for each language
for lang_code, lang_label in languages:
    continuation_token = None
    prev_length = len(all_reviews)
    
    while True:
        result, continuation_token = reviews(
            app_id,
            lang=lang_code,
            sort=Sort.NEWEST,
            count=100,
            continuation_token=continuation_token
        )
        
        # Add language label to each review
        for review in result:
            review['language'] = lang_label
        
        all_reviews.extend(result)
        
        # Break if no more reviews or if number of reviews isn't increasing
        current_length = len(all_reviews)
        if not continuation_token or current_length - prev_length < 100:
            break
            
        prev_length = current_length

# Convert all reviews into a pandas DataFrame
df = pd.DataFrame(all_reviews)

print("\nReview Statistics:")
print("=" * 50)
print(f"Total number of reviews collected: {len(df)}")
print("\nBreakdown by language:")
print("-" * 50)
language_counts = df['language'].value_counts()
print(language_counts)
print("-" * 50)
print(f"Number of languages with reviews: {len(language_counts)}")
# Inspect the first few rows
df.head()


Review Statistics:
Total number of reviews collected: 18350

Breakdown by language:
--------------------------------------------------
language
German        4914
English       4617
French        1711
Italian       1315
Dutch          974
Spanish        967
Polish         641
Portuguese     576
Russian        473
Romanian       286
Swedish        261
Norwegian      205
Japanese       201
Finnish        175
Czech          158
Greek          117
Hungarian      114
Danish         102
Thai            99
Turkish         65
Croatian        65
Slovak          58
Slovenian       53
Chinese         47
Bulgarian       43
Arabic          32
Serbian         21
Ukrainian       14
Lithuanian      12
Estonian         9
Indonesian       7
Latvian          7
Korean           4
Hebrew           4
Malay            2
Persian          1
Name: count, dtype: int64
--------------------------------------------------
Number of languages with reviews: 36


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,language
0,d3f785ff-3bf8-49da-9313-6763ffeacf2a,ALEX TEO,https://play-lh.googleusercontent.com/a-/ALV-U...,BEWARE!! absolutely useless. Everytime i locke...,1,0,5.3.4,2025-04-09 15:13:37,None,NaT,5.3.4,English
1,2bedd865-d385-45e2-83f8-bbb3317587ba,Jean Richards,https://play-lh.googleusercontent.com/a/ACg8oc...,my bmw is just best super to drive so comforta...,5,0,None,2025-04-09 14:54:20,None,NaT,None,English
2,54d14726-aa2d-4587-8647-b50b3bb316e9,Imran Ali Jamal,https://play-lh.googleusercontent.com/a/ACg8oc...,Great app for keeping track of your BMW.,4,0,5.3.3,2025-04-09 13:32:29,None,NaT,5.3.3,English
3,73532596-6628-4b66-ab82-14af6d3f29ab,KENNETH ORJI,https://play-lh.googleusercontent.com/a-/ALV-U...,I love the fact that it is free to download an...,5,0,5.3.3,2025-04-09 11:24:57,None,NaT,5.3.3,English
4,3d7d6de6-1f01-42f2-865c-49124326b268,tihomir jarnjevic,https://play-lh.googleusercontent.com/a/ACg8oc...,"no limit for charging, or timer",4,0,5.3.3,2025-04-09 07:46:06,None,NaT,5.3.3,English


In [4]:
import os
import time
import json
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import subprocess
import atexit
import signal
import warnings

# Suppress the pandas FutureWarning about concatenation
warnings.filterwarnings('ignore', category=FutureWarning)

def translate_text(text, source_lang, model_name):
    """
    Translate text to English using Ollama with an enhanced prompt.
    """
    prompt = f"""You are a professional translator specialized in automotive app reviews. Translate the following review written in {source_lang} into English.

TASK: Translate this BMW app review accurately while preserving:
- The original tone and sentiment
- Technical terminology (e.g., Connected Drive, MyBMW App, iDrive, Digital Key)
- App-specific or BMW-specific expressions and informal language

Please observe the following guidelines:
1. Preserve technical terms as given.
2. Use consistent terminology (e.g., "Ladestation/Borne de recharge" → "charging station").
3. If text is unclear, translate literally rather than interpreting.
4. Keep numbers, percentages, units, error codes, emojis, and model numbers intact.

IMPORTANT:
- Return only the translated text, do not add any additional information, explanations, or comments.
- Maintain the original paragraph structure and tone.
- Do not extend or elaborate beyond what is in the original text.

Review:
"{text}"

Translation:"""
    
    process = subprocess.run(
        ["ollama", "run", model_name],
        input=prompt,
        text=True,
        capture_output=True
    )
    
    return process.stdout.strip()

def translate_all_reviews(df, model_name, base_dir="bmw_app_analysis", checkpoint_interval=100):
    """
    Translate all non-English reviews in a DataFrame to English with robust checkpointing.
    Saves English reviews as batch000.csv, then each batch of 100 translations as separate files.
    
    Args:
        df: DataFrame containing reviews with 'content' and 'language' columns
        model_name: Name of the Ollama model to use
        base_dir: Base directory for saving files
        checkpoint_interval: Save intermediate results after this many translations
    
    Returns:
        DataFrame with all reviews and added 'content_english' column
    """
    # Create a single translations directory for all files
    translations_dir = os.path.join(base_dir, "translations")
    os.makedirs(translations_dir, exist_ok=True)
    
    # Set up logging
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_id = f"run_{timestamp}"  # Keep a run ID for the progress file
    progress_file = os.path.join(translations_dir, "progress.json")
    
    # Variables to track state for autosave
    _current_batch_df = pd.DataFrame()
    _combined_df = None  # For emergency saves only
    _translated_indices = []
    _last_checkpoint_num = 0
    _progress_pct = 0
    
    # Function to save current progress (for emergency saves)
    def save_current_progress(signal_received=None, frame=None):
        nonlocal _current_batch_df, _combined_df, _translated_indices, _last_checkpoint_num, _progress_pct
        
        if len(_translated_indices) == 0:
            print("\nNo progress to save.")
            return
            
        # If we have unsaved translations in the current batch, save them
        if not _current_batch_df.empty:
            emergency_file = os.path.join(translations_dir, f"emergency_{_last_checkpoint_num+1:03d}.csv")
            _current_batch_df.to_csv(emergency_file, index=False)
            
            # Also save a combined file for recovery
            if _combined_df is not None:
                full_emergency = os.path.join(translations_dir, "emergency_all.csv")
                _combined_df.to_csv(full_emergency, index=False)
                print(f"Full emergency backup saved to: {full_emergency}")
            
        # Update progress file
        progress_data = {
            'run_id': run_id,
            'model_name': model_name,
            'total_reviews': len(df),
            'english_reviews': len(english_df) if 'english_df' in locals() else 0,
            'non_english_reviews': len(non_english_df) if 'non_english_df' in locals() else 0,
            'translated_reviews': len(_translated_indices),
            'progress_percent': _progress_pct,
            'checkpoint_number': _last_checkpoint_num + 1,
            'last_checkpoint': emergency_file if '_current_batch_df' in locals() and not _current_batch_df.empty else None,
            'translated_indices': [str(i) for i in _translated_indices],
            'status': 'paused'
        }
        
        with open(progress_file, 'w') as f:
            json.dump(progress_data, f, indent=4)
        
        print(f"\nEmergency progress saved.")
        print(f"Progress: {len(_translated_indices)} reviews translated ({_progress_pct}%)")
        print("You can resume translation later by running the function again.")
        
        if signal_received:
            exit(0)
    
    # Register handlers for various exit scenarios
    atexit.register(save_current_progress)
    signal.signal(signal.SIGINT, save_current_progress)  # Ctrl+C
    signal.signal(signal.SIGTERM, save_current_progress)  # Termination signal
    
    # Create a working copy of the DataFrame
    working_df = df.copy()
    
    # Ensure 'content_english' column exists
    if 'content_english' not in working_df.columns:
        working_df['content_english'] = working_df['content']
    
    # Separate English and non-English reviews
    english_df = working_df[working_df['language'] == 'English'].copy()
    non_english_df = working_df[working_df['language'] != 'English'].copy()
    
    print(f"Total reviews: {len(working_df)}")
    print(f"English reviews: {len(english_df)}")
    print(f"Non-English reviews: {len(non_english_df)}")
    
    # Save English-only reviews with simple name: batch000.csv
    english_checkpoint = os.path.join(translations_dir, "batch000.csv")
    english_df.to_csv(english_checkpoint, index=False)
    print(f"Saved English-only reviews as: {english_checkpoint}")
    
    # Create a combined DataFrame for tracking progress
    combined_df = english_df.copy()  # Start with English reviews
    _combined_df = combined_df  # Copy for emergency saves
    
    # Check for existing progress
    last_checkpoint_num = 0
    translated_indices = []
    
    if os.path.exists(progress_file):
        try:
            with open(progress_file, 'r') as f:
                progress_data = json.load(f)
                last_checkpoint_num = progress_data.get('checkpoint_number', 0)
                translated_indices = [int(idx) for idx in progress_data.get('translated_indices', [])]
                
                if translated_indices:
                    print(f"Found previous progress: {len(translated_indices)}/{len(non_english_df)} reviews translated")
                    print(f"Last checkpoint: {last_checkpoint_num}")
                    
                    resume = input("Resume from last checkpoint? (y/n): ")
                    if resume.lower() == 'y':
                        # Load all existing batch files and reconstruct the combined DataFrame
                        print("Loading existing batches...")
                        combined_df = english_df.copy()  # Start with English reviews
                        
                        # Load each numbered batch
                        for i in range(1, last_checkpoint_num + 1):
                            batch_file = os.path.join(translations_dir, f"batch{i:03d}.csv")
                            if os.path.exists(batch_file):
                                batch = pd.read_csv(batch_file)
                                print(f"Loading batch{i:03d}.csv ({len(batch)} reviews)")
                                combined_df = pd.concat([combined_df, batch])
                        
                        _combined_df = combined_df  # Copy for emergency saves
                        print(f"Loaded {len(combined_df) - len(english_df)} translated reviews from checkpoints")
                    else:
                        print("Starting fresh, but keeping English-only checkpoint")
                        translated_indices = []
                        last_checkpoint_num = 0
        except Exception as e:
            print(f"Error reading progress file: {e}")
            translated_indices = []
    
    # Update global variables for emergency saves
    _translated_indices = translated_indices
    _last_checkpoint_num = last_checkpoint_num
    
    # Get remaining reviews to translate
    remaining_indices = [idx for idx in non_english_df.index if idx not in translated_indices]
    print(f"Translating {len(remaining_indices)} remaining reviews...")
    
    # Option to stop before starting (in case they loaded the wrong checkpoint)
    if remaining_indices:
        proceed = input("Proceed with translation? (y/n): ")
        if proceed.lower() != 'y':
            print("Translation canceled. All loaded data is preserved.")
            return combined_df
    
    # Main translation loop
    try:
        # Process reviews in batches
        total_batches = (len(remaining_indices) + checkpoint_interval - 1) // checkpoint_interval
        
        for batch_idx in range(total_batches):
            next_checkpoint_num = last_checkpoint_num + 1
            
            print(f"\n========== BATCH {next_checkpoint_num:03d} ==========")
            print(f"Processing reviews {batch_idx * checkpoint_interval + 1} to {min((batch_idx + 1) * checkpoint_interval, len(remaining_indices))}")
            
            # Get the indices for this batch
            start_idx = batch_idx * checkpoint_interval
            end_idx = min((batch_idx + 1) * checkpoint_interval, len(remaining_indices))
            batch_size = end_idx - start_idx
            batch_indices = remaining_indices[start_idx:end_idx]
            
            # Create batch DataFrame
            batch_df = pd.DataFrame(columns=working_df.columns)
            _current_batch_df = batch_df  # Copy for emergency saves
            
            # Process reviews in this batch
            progress_bar = tqdm(
                total=batch_size,
                desc=f"Batch {next_checkpoint_num:03d}",
                ncols=100,
                bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
            )
            
            batch_translated = 0
            
            for idx in batch_indices:
                # Get review text and language
                original_text = working_df.loc[idx, 'content']
                source_lang = working_df.loc[idx, 'language']
                
                # Skip if empty
                if pd.isna(original_text) or not original_text.strip():
                    progress_bar.update(1)
                    continue
                
                # Translate the text
                translated_text = translate_text(original_text, source_lang, model_name)
                
                # Create a row to add 
                row = working_df.loc[[idx]].copy()
                row['content_english'] = translated_text
                
                # Add row to the batch DataFrame (using a method that avoids the warning)
                if batch_df.empty:
                    batch_df = row.copy()
                else:
                    batch_df = pd.concat([batch_df, row], ignore_index=False)
                
                # Update for emergency saves
                _current_batch_df = batch_df
                
                # Add to translated indices
                translated_indices.append(idx)
                _translated_indices = translated_indices
                batch_translated += 1
                
                # Update progress bar
                progress_bar.update(1)
                
                # Show occasional status updates within the batch
                if batch_translated % 10 == 0:
                    progress_pct = round(len(translated_indices) / len(non_english_df) * 100, 1)
                    _progress_pct = progress_pct
                    progress_bar.set_postfix({"Total": f"{len(translated_indices)}/{len(non_english_df)}", "Progress": f"{progress_pct}%"})
            
            # Close progress bar for this batch
            progress_bar.close()
            
            # Add batch to combined DataFrame (for tracking only)
            combined_df = pd.concat([combined_df, batch_df])
            _combined_df = combined_df  # Copy for emergency saves
            
            # Calculate progress percentage
            progress_pct = round(len(translated_indices) / len(non_english_df) * 100, 1)
            _progress_pct = progress_pct  # Update for emergency saves
            
            # Save ONLY THIS BATCH with simple name: batch001.csv, batch002.csv, etc.
            checkpoint_file = os.path.join(translations_dir, f"batch{next_checkpoint_num:03d}.csv")
            batch_df.to_csv(checkpoint_file, index=False)
            
            # Update progress file
            progress_data = {
                'run_id': run_id,
                'model_name': model_name,
                'total_reviews': len(working_df),
                'english_reviews': len(english_df),
                'non_english_reviews': len(non_english_df),
                'translated_reviews': len(translated_indices),
                'progress_percent': progress_pct,
                'checkpoint_number': next_checkpoint_num,
                'last_checkpoint': checkpoint_file,
                'translated_indices': [str(i) for i in translated_indices],
                'status': 'in_progress'
            }
            
            with open(progress_file, 'w') as f:
                json.dump(progress_data, f, indent=4)
            
            # Print batch summary
            print(f"\nBatch {next_checkpoint_num:03d} complete!")
            print(f"Saved batch with {len(batch_df)} translations: {checkpoint_file}")
            print(f"Overall progress: {len(translated_indices)}/{len(non_english_df)} reviews ({progress_pct}%)")
            
            # Show sample translations
            if not batch_df.empty:
                print("\nSample translations from this batch:")
                sample_count = min(3, len(batch_df))
                sample_indices = batch_df.index[-sample_count:]
                
                for idx in sample_indices:
                    lang = batch_df.loc[idx, 'language']
                    orig = batch_df.loc[idx, 'content']
                    trans = batch_df.loc[idx, 'content_english']
                    print(f"\n[{lang}] Original: {orig[:100]}..." if len(orig) > 100 else f"\n[{lang}] Original: {orig}")
                    print(f"[English] Translation: {trans[:100]}..." if len(trans) > 100 else f"[English] Translation: {trans}")
                    print("---")
            
            # Update tracking variables for next batch
            last_checkpoint_num = next_checkpoint_num
            _last_checkpoint_num = last_checkpoint_num  # Update for emergency saves
            
            # Ask to continue if not the last batch
            if batch_idx < total_batches - 1:
                continue_translation = input("\nContinue to next batch? (y/n): ")
                if continue_translation.lower() != 'y':
                    # Update progress status to paused
                    progress_data['status'] = 'paused'
                    with open(progress_file, 'w') as f:
                        json.dump(progress_data, f, indent=4)
                    
                    print(f"\nTranslation paused at {progress_pct}% complete.")
                    print(f"To resume later, run the function again and select 'y' when prompted to resume.")
                    return combined_df
    
    except KeyboardInterrupt:
        print("\nTranslation interrupted by user")
        if 'progress_bar' in locals() and progress_bar is not None:
            progress_bar.close()
        # The autosave handler will take care of saving progress
        return combined_df
    
    # Translation complete - save final merged result
    print("\nMerging all batches into final file...")
    
    # Start with just English reviews
    final_df = english_df.copy()
    
    # Load and merge all batch files
    for i in range(1, last_checkpoint_num + 1):
        batch_file = os.path.join(translations_dir, f"batch{i:03d}.csv")
        if os.path.exists(batch_file):
            batch = pd.read_csv(batch_file)
            print(f"Adding batch{i:03d}.csv ({len(batch)} reviews)")
            final_df = pd.concat([final_df, batch])
    
    final_file = os.path.join(translations_dir, "final_translated.csv")
    final_df.to_csv(final_file, index=False)
    
    # Update progress file
    progress_data = {
        'run_id': run_id,
        'model_name': model_name,
        'total_reviews': len(working_df),
        'english_reviews': len(english_df),
        'non_english_reviews': len(non_english_df),
        'translated_reviews': len(translated_indices),
        'progress_percent': 100,
        'status': 'completed',
        'checkpoint_number': last_checkpoint_num,
        'batch_count': last_checkpoint_num,
        'final_output': final_file,
        'translated_indices': [str(i) for i in translated_indices]
    }
    
    with open(progress_file, 'w') as f:
        json.dump(progress_data, f, indent=4)
    
    print(f"\nTranslation complete! All {len(translated_indices)} non-English reviews translated")
    print(f"Final merged output saved to: {final_file}")
    print(f"Final file contains {len(final_df)} total reviews (English + translated)")
    
    return final_df

# Helper function to merge all checkpoint files
def merge_translation_batches(base_dir="bmw_app_analysis"):
    """
    Merge all translation batch files into a single DataFrame.
    """
    translations_dir = os.path.join(base_dir, "translations")
    
    # Ensure the directory exists
    if not os.path.exists(translations_dir):
        print(f"Error: Translations directory {translations_dir} not found!")
        return None
        
    # Start with English reviews
    english_file = os.path.join(translations_dir, "batch000.csv")
    if not os.path.exists(english_file):
        print(f"Error: English file {english_file} not found!")
        return None
        
    merged_df = pd.read_csv(english_file)
    print(f"Loaded English reviews: {len(merged_df)}")
    
    # Find all batch files and sort them numerically
    batch_files = [f for f in os.listdir(translations_dir) if f.startswith("batch") and f != "batch000.csv"]
    batch_files.sort(key=lambda x: int(x.replace("batch", "").replace(".csv", "")))
    
    # Merge each batch
    for batch_file in batch_files:
        file_path = os.path.join(translations_dir, batch_file)
        batch_df = pd.read_csv(file_path)
        print(f"Adding {batch_file} ({len(batch_df)} reviews)")
        merged_df = pd.concat([merged_df, batch_df])
    
    # Save the merged result
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    merged_file = os.path.join(translations_dir, f"merged_translations_{timestamp}.csv")
    merged_df.to_csv(merged_file, index=False)
    
    print(f"Merged all batches successfully!")
    print(f"Total reviews: {len(merged_df)}")
    print(f"Saved to: {merged_file}")
    
    return merged_df

In [ ]:
# Execute the translation
df_translated = translate_all_reviews(df, ollama_model_name, checkpoint_interval=1000)

# Print dimensions of the dataframe before and after translation
print(f"Original DataFrame dimensions: {df.shape} (rows, columns)")
print(f"Translated DataFrame dimensions: {df_translated.shape} (rows, columns)")

Total reviews: 18350
English reviews: 4617
Non-English reviews: 13733
Saved English-only reviews as: bmw_app_analysis/translations/batch000.csv
Found previous progress: 7000/13733 reviews translated
Last checkpoint: 7
Loading existing batches...
Loading batch001.csv (1000 reviews)
Loading batch002.csv (1000 reviews)
Loading batch003.csv (1000 reviews)
Loading batch004.csv (1000 reviews)
Loading batch005.csv (1000 reviews)
Loading batch006.csv (1000 reviews)
Loading batch007.csv (1000 reviews)
Loaded 7000 translated reviews from checkpoints
Translating 6733 remaining reviews...

========== BATCH 008 ==========
Processing reviews 1 to 1000


Batch 008: 100%|███████████████████████████████████████████████████████████| 1000/1000 [31:46<00:00]



Batch 008 complete!
Saved batch with 1000 translations: bmw_app_analysis/translations/batch008.csv
Overall progress: 8000/13733 reviews (58.3%)

Sample translations from this batch:

[Spanish] Original: buena app para saber cómo está tu vehículo
[English] Translation: "Good app for knowing how your vehicle is."
---

[Spanish] Original: Fácil de manejar, intuitiva, controlas muchos aspectos del automóvil desde el celular
[English] Translation: Easy to use, intuitive, you control many aspects of the car from your cell phone.
---

[Spanish] Original: esta muy bien, tienes información de tu bmw todo el tiempo
[English] Translation: This is very good, you have information about your BMW all the time.
---

========== BATCH 009 ==========
Processing reviews 1001 to 2000


Batch 009:  14%|████████▍                                                   | 141/1000 [03:22<19:35]